In [149]:
import spacy
from spacy.lang.ru import Russian

import medspacy
from medspacy.ner import TargetRule
from medspacy.context import ConTextRule, ConTextComponent, ConTextItem
from medspacy.visualization import visualize_dep, visualize_ent

import pymorphy2

import warnings
warnings.filterwarnings('ignore')

In [150]:
text = "У людей с сахарным диабетом 2 типа часто сначала нет симптомов. На основании результатов обследования сахарный диабет второго типа может быть исключен."

In [151]:
text = " .".join(text.split('.'))

In [152]:
morph = pymorphy2.MorphAnalyzer()
new_txt = " ".join([morph.parse(word)[0].normal_form for word in text.split()])

In [153]:
new_txt

'у человек с сахарный диабет 2 тип часто сначала нет симптом . на основание результат обследование сахарный диабет второй тип мочь быть исключить .'

In [154]:
nlp = medspacy.load()
doc = nlp(new_txt)

In [155]:
nlp.pipe_names

['medspacy_pyrush', 'medspacy_target_matcher', 'medspacy_context']

In [156]:
context = nlp.get_pipe("medspacy_context")
context.categories

{'FAMILY',
 'HISTORICAL',
 'HYPOTHETICAL',
 'NEGATED_EXISTENCE',
 'POSSIBLE_EXISTENCE'}

In [157]:
target_matcher = nlp.get_pipe("medspacy_target_matcher")

In [158]:
target_rules = [
    TargetRule("диабет X тип", "CONDITION",
              pattern=[
                        {"LOWER": 'диабет'},
                        {"LOWER": {"IN": ["2", "ii", "второй"]}},
                        {"LOWER": 'тип'}]),
    TargetRule("симптом", "CONDITION"),
]

In [159]:
target_matcher.add(target_rules)

In [160]:
doc = nlp(new_txt)

In [161]:
visualize_ent(doc)

In [162]:
context = ConTextComponent(nlp, rules=None)
context_rules = [
    ConTextRule(literal="нет", 
                category="NEGATED_EXISTENCE", 
                direction="FORWARD"),
    ConTextRule(literal="исключить", 
                category="NEGATED_EXISTENCE", 
                direction="BACKWARD"),
    ConTextRule(literal='.', 
                category="CONJ", 
                direction="TERMINATE")
]

context.add(context_rules)
context(doc)

у человек с сахарный диабет 2 тип часто сначала нет симптом . на основание результат обследование сахарный диабет второй тип мочь быть исключить .

In [163]:
visualize_ent(doc)

In [164]:
visualize_dep(doc)

In [165]:
for ent in doc.ents:
    if ent._.is_negated:
        print(ent)

симптом
диабет второй тип
